# Reshaping Exercises

Before diving into actually doing some exercises, a few notes about reshaping in `pandas`. 

**So many tools:** As is so often the case with `pandas`, when there are multiple ways that something can be done, `pandas` implements all of them, even it if results in a lot of duplication of funcationality. As a result, there are several sets of methods that do very similar things, and which can be hard to keep straights: 

- `melt` and `pivot`: Tools for moving a dataset from long-to-wide (`pivot`) or wide-to-long (`melt`).
- `pivot_table`: Like `pivot`, but allows you to do aggregations during the reshape if you want.
- `stack` and `unstack`: Like `melt` and `pivot`, but only for use when your variables are stored in indices. I don't use these much. 

Personally, **I find melt and pivot most useful.** `pivot_table` *can* do aggregations during the reshape, but you can always do aggregations yourself using `groupby`. :) 

Also, I have to go look at [these pictures](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html)... every time I reshape. Don't hesistate to look them up for help!

**What SHOULD I want my data to look like?** Just a reminder that a natural question to ask is "in what format should I want my data?" Obviously this question is subjective, but as you read in the Hadley Wickham [piece I assigned](http://vita.had.co.nz/papers/tidy-data.html), this is a good standard definition for a "tidy" dataset to which we might aspire: 

1. Each variable forms a column.
2. Each observation forms a row.
3. Each type of observational unit forms a table.

In general, I'm a big fan of this approach, and I think it's a good one to use (especially numbers 1 and 2).

Note that the question of what forms an "observation" depends on the analysis you plan to run (which depends on the answer you seek to ask! See! EVERYTHING ALWAYS COMES BACK TO THE QUESTION YOU ARE SEEKING TO ANSWER!). Suppose you have data on GDP from different countries for many years. If you plan to run a panel analysis where each observation is a country-year, then each row should be a country-year. But if you plan to do a country-level analysis using measures of, say, GDP variation over time, then each row should be a country!



## Exercises!

So in this exercise, we're going to work with data from the World Development Indicators. Unlike in previous exercises, however, where I cleaned up the data before giving it to you, we'll be working with the raw data just as you would get it from the World Bank download site. 


## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_reshaping.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex2_num_rows",
    "ex4_num_rows",
    "ex5_num_countries",
    "ex6_relationship",
    "ex7_relationship",
    "ex8_countries_in_panel",
    "ex10_demeaned_corr",
}
```


### Submission Limits

Please remember that you are **only allowed three submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.

### Exercise 1

Import the data using a link from [here](https://github.com/nickeubank/MIDS_Data/blob/master/World_Development_Indicators/raw_WDI_Data_csv.zip) and load it into pandas without unzipping it (did you know pandas will read zip files directly?!).

In [37]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

wdi = pd.read_csv(
    "https://github.com/nickeubank/MIDS_Data/raw/refs/heads/master/World_Development_Indicators/raw_WDI_Data_csv.zip",
    delimiter=",",
)
wdi.sample(20)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
13873,Europe & Central Asia,ECS,"Population ages 0-14, female",SP.POP.0014.FE.IN,9.176300e+07,9.333736e+07,9.458645e+07,9.553158e+07,9.629630e+07,9.692441e+07,...,7.526162e+07,7.560222e+07,7.608784e+07,7.670580e+07,7.740251e+07,7.812596e+07,7.878307e+07,7.935797e+07,7.985351e+07,NaN
32423,Late-demographic dividend,LTE,Exports of goods and services (constant 2010 US$),NE.EXP.GNFS.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,4.307811e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88872,Armenia,ARM,"Net incurrence of liabilities, total (% of GDP)",GC.LBL.TOTL.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,3.604177e+00,2.826362e+00,2.807737e+00,3.143894e+00,1.734749e+00,6.281278e+00,8.079014e+00,5.009026e+00,NaN,NaN
253156,Lesotho,LSO,GDP (constant LCU),NY.GDP.MKTP.KN,1.781631e+09,1.814855e+09,2.093105e+09,2.315289e+09,2.506326e+09,2.560315e+09,...,1.940657e+10,2.074589e+10,2.199024e+10,2.239569e+10,2.309482e+10,2.373377e+10,2.448827e+10,2.392839e+10,2.428093e+10,NaN
130508,Cameroon,CMR,Nitrous oxide emissions in energy sector (thou...,EN.ATM.NOXE.EG.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14135,Europe & Central Asia,ECS,Rural population,SP.RUR.TOTL,2.962344e+08,2.960622e+08,2.959232e+08,2.955538e+08,2.950779e+08,2.943630e+08,...,2.594151e+08,2.585017e+08,2.579926e+08,2.576801e+08,2.573846e+08,2.570192e+08,2.565680e+08,2.558867e+08,2.550628e+08,NaN
279845,"Micronesia, Fed. Sts.",FSM,"Adjusted net enrollment rate, primary, female ...",SE.PRM.TENR.FE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,8.829697e+01,8.539293e+01,NaN,NaN,NaN,NaN
43017,Low income,LIC,"Tariff rate, applied, weighted mean, all produ...",TM.TAX.MRCH.WM.AR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,9.200000e+00,7.890000e+00,8.870000e+00,9.280000e+00,8.210000e+00,9.070000e+00,7.390000e+00,9.790000e+00,NaN,NaN
386264,Togo,TGO,"Net bilateral aid flows from DAC donors, Unite...",DC.DAC.GBRL.CD,NaN,NaN,NaN,NaN,2.000000e+04,1.000000e+04,...,-8.000000e+04,1.920000e+06,5.000000e+04,NaN,NaN,5.000000e+04,NaN,NaN,NaN,NaN
365484,St. Kitts and Nevis,KNA,"Net financial flows, bilateral (NFL, current US$)",DT.NFL.BLAT.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,-6.568000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
wdi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422136 entries, 0 to 422135
Data columns (total 64 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Country Name    422136 non-null  object 
 1   Country Code    422136 non-null  object 
 2   Indicator Name  422136 non-null  object 
 3   Indicator Code  422136 non-null  object 
 4   1960            38261 non-null   float64
 5   1961            41976 non-null   float64
 6   1962            44155 non-null   float64
 7   1963            44035 non-null   float64
 8   1964            44563 non-null   float64
 9   1965            47249 non-null   float64
 10  1966            46846 non-null   float64
 11  1967            48578 non-null   float64
 12  1968            48157 non-null   float64
 13  1969            49960 non-null   float64
 14  1970            92948 non-null   float64
 15  1971            99195 non-null   float64
 16  1972            101947 non-null  float64
 17  1973      

### Exercise 2

In this exercise, we'd like to look at the relationship between GDP and inequality across countries. So before anything else, thin down your dataset by getting only the rows related to:

- GDP per capita (in Constant US Dollars -- the "constant" means values are have been adjusted for inflation, so values from 1970 can be directly compared with 2010)
- Gini coefficient
- Information on income shares (what percentage of total income goes to X% of the income distribution). **There are 7 of these in total.**

Store the number of rows you get from this updated dataset in `"ex2_num_rows"`.

**NOTE:** this is non-trivial. This is a big dataset with LOTS of variables, so you'll have to use your data wrangling skills to see all the variables names. Remember that if you want to see all the entries in something but python compresses it (so it doesn't take up too much space), you can loop over it and print each item separately to make it show everything!

**NOTE 2:** If you _do_ print out all the variable names, please comment out the code before you submit your notebook so graders don't end up swimming in print-outs. 

In [4]:
# wdi.columns.tolist()
wdi.sample(1)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
410753,Vanuatu,VUT,"Share of youth not in education, employment or...",SL.UEM.NEET.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
wdi["Indicator Name"].unique()

# Look for GDP per capita variables
# wdi[wdi["Indicator Name"].str.contains("GDP per capita", case=False, na=False)][
# "Indicator Name"
# ].unique()

# Look for Gini variables
wdi[wdi["Indicator Name"].str.contains("Gini", case=False, na=False)][
    "Indicator Name"
].unique()

# Look for income share variables
# wdi[wdi['Indicator Name'].str.contains("income share", case=False, na=False)]['Indicator Name'].unique()

array(['GINI index (World Bank estimate)'], dtype=object)

In [33]:
target_indicators = [
    "GDP per capita (constant 2010 US$)",
    "GINI index (World Bank estimate)",
    "Income share held by lowest 10%",
    "Income share held by lowest 20%",
    "Income share held by second 20%",
    "Income share held by third 20%",
    "Income share held by fourth 20%",
    "Income share held by highest 20%",
    "Income share held by highest 10%",
]

wdi_ex2 = wdi[wdi["Indicator Name"].isin(target_indicators)].copy()

results = {}
results["ex2_num_rows"] = len(wdi_ex2)
print(results)

{'ex2_num_rows': 2376}


In [ ]:
wdi_ex2.head()
wdi_ex2.sample(20)
# wdi_ex2["Indicator Code"].unique()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
275568,Mauritania,MRT,GINI index (World Bank estimate),SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,32.600000,NaN,NaN,NaN,NaN,NaN
7051,East Asia & Pacific,EAS,Income share held by fourth 20%,SI.DST.04TH.20,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
657,Arab World,ARB,Income share held by highest 20%,SI.DST.05TH.20,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146166,"Congo, Dem. Rep.",COD,Income share held by highest 20%,SI.DST.05TH.20,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,48.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411601,"Venezuela, RB",VEN,Income share held by lowest 10%,SI.DST.FRST.10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267693,Malaysia,MYS,Income share held by second 20%,SI.DST.02ND.20,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.300000,NaN,10.100000,NaN,10.10000,NaN,NaN,NaN,NaN
334849,Rwanda,RWA,Income share held by lowest 10%,SI.DST.FRST.10,NaN,NaN,NaN,NaN,NaN,NaN,...,2.300000,NaN,NaN,2.400000,NaN,NaN,2.400000,NaN,NaN,NaN
87004,Argentina,ARG,Income share held by lowest 10%,SI.DST.FRST.10,NaN,NaN,NaN,NaN,NaN,NaN,...,1.600000,1.700000,1.700000,1.800000,1.800000,NaN,1.800000,1.900000,NaN,NaN
5317,Early-demographic dividend,EAR,GDP per capita (constant 2010 US$),NY.GDP.PCAP.KD,1138.120007,1160.43403,1181.382407,1215.121629,1284.659152,1321.543607,...,2938.879732,3039.456791,3122.614352,3210.941286,3293.520608,3400.04993,3515.391010,3632.193253,3732.285934,NaN
37432,Latin America & the Caribbean (IDA & IBRD coun...,TLA,Income share held by fourth 20%,SI.DST.04TH.20,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Exercise 3

OK, that makes our data a lot more manageable, but it's still pretty heinous.

In our analysis, we know that we'll want to have country-years as our units of observation, so let's start reshaping. Come up with a strategy that gets our inequality variables as columns, and makes each of our future observations (i.e. each country-year) a row. 

*Hint:* It can sometimes be helpful to drop useless columns before a reshape.

*Other hint:* Sometimes you will end up with your column names stored in a hierarchical (multi-level) `.column` object structure. As with hierarchical indices for rows, they’re a pain. (Whether this occurs depends on the exact syntax you use, so you may or may not get it).

The two easy ways to fix these hierarchical `.column` objects are:

- Just drop a level: df.columns.drop_level(0)` where 0 is which level you want to drop.
- Directly manipulate the column names by doing something to the df.columns object and re-assigning to df.columns. For example, you can just concatenate two columns levels (they're stored as tuples) with a dash between them with `df.columns = map(lambda x: str(x[0]) + "-" + str(x[1]), df.columns)`.

In [ ]:
year_cols = wdi_ex2.columns[wdi_ex2.columns.str.fullmatch(r"\d{4}")]
keep = ["Country Name", "Country Code", "Indicator Name"] + list(year_cols)
tidy = wdi_ex2[keep].copy()

wdi_long = tidy.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name"],
    value_vars=year_cols,
    var_name="Year",
    value_name="Indicator",
)
wdi_long["Year"] = wdi_long["Year"].astype(int)

country_year = wdi_long.pivot_table(
    index=["Country Name", "Country Code", "Year"],
    columns="Indicator Name",
    values="Indicator",
    aggfunc="first",
).sort_index()

country_year = country_year.reset_index()

country_year.sample(20)

Indicator Name,Country Name,Country Code,Year,GDP per capita (constant 2010 US$),GINI index (World Bank estimate),Income share held by fourth 20%,Income share held by highest 10%,Income share held by highest 20%,Income share held by lowest 10%,Income share held by lowest 20%,Income share held by second 20%,Income share held by third 20%
7508,Mozambique,MOZ,2008,401.735837,45.6,20.1,36.8,51.4,1.9,5.2,9.5,13.7
7216,Middle East & North Africa (excluding high inc...,MNA,1998,3074.685364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9628,South Asia,SAS,1960,332.069310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9231,Senegal,SEN,1989,1110.703474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3837,Germany,DEU,2004,38673.888113,30.4,22.4,24.2,39.1,3.4,8.5,13.1,16.9
5097,"Iran, Islamic Rep.",IRN,1985,4773.002204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3194,Ethiopia,ETH,2012,389.938967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6156,Lebanon,LBN,1999,5783.972027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9207,Senegal,SEN,1965,1306.389660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4842,IDA total,IDA,1966,682.436497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Exercise 4

Now that you've done your reshapes, subset for observations that are post-1980. Store the number of rows in your reshaped data with only data from the years after 1980 in `"ex4_num_rows"`.

### Exercise 5

Subset for observations for which we have GINI index estimates. How many unique *countries* do you end up with? Store this answer in `ex5_num_countries`.

### Exercise 6

Plot the relationship between **log** GDP per capita and Gini scores for all country years and add a linear fit as well. Is higher inequality associated with higher log GDP? Store your answer (as `"yes"` or `"no"`) in `results` under the key `"ex6_relationship"`.

### Exercise 7

It's important to not always rely too much on a single measure of something like inequality. To check the robustness of our result, let's be sure to also look at the relationship between inequality and income by using the income share of the richest 10% instead of just the Gini coefficient. Plot that relationship.

Do you also find that income inequality (as measured by the share of income accruing to the top 10% of earners) is positively or negatively correlated with gdp per capita? Store your answer in your `results` dict under the key `"ex7_relationship"` as the string `"positively correlated"` or the string `"negatively correlated"`.

### Exercise 8

This analysis has allowed us to evaluate whether richer countries tend to have greater income inequality, but it certainly has not established whether we should expect that as countries become wealthier, their income distribution will also become more even. In other words, we have established a correlation between income inequality and wealth, but we certainly have not established a causal relationship between wealth and income inequality.

One way we might be able to get better traction on this question is to look at whether, as individual countries become wealthier, their income distribution also becomes more even.

To answer this question, let's begin by subsetting our data to the years 2005 to 2015 (inclusive)—the period in which the largest number of countries in our data have inequality data. Then also subset for the set of countries with Gini data for all years in this period. This gives us what's called a *balanced panel*—a dataset in which we have data on entities *at a number of different points in time* (a panel dataset) and where at every point in time, we have data for the *same set of entities* (a balanced panel dataset). 

Balanced panels are very desirable because the fact that they contain the same entities at all moments in time means that any changes we measure over time are the result of real changes in the outcomes we care about and *not* the result of changes in the composition of our sample.

Store the number of countries still in your data in `"ex8_countries_in_panel"`.

### Exercise 9

Now, demean both Gini coefficient scores and log GDP per capita for all countries. In other words, for each country, calculate that country's average Gini score over this period and subtract that average from all of the country's Gini scores, then do the same for log GDP per capita. Then plot these countries' *demeaned* Gini scores against demeaned log GDP per capita.

Essentially, this is a way of looking at whether, when countries are richer than their average during this period, they have more equal income distributions than on average.

### Exercise 10 

Calculate the correlation between these two demand outcomes in our balanced panel and store the result in `"ex10_demeaned_corr"`.